In [1]:
import tensorflow as tf
print(tf.__version__)
import cv2
print(cv2.__version__)
import time
import os
import numpy as np
import posenet
from collections import deque

c:\users\micha\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.12.0
3.4.5


# Function for calculating angle between two vectors

In [2]:

def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle(v1, v2):
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return 180*(np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0)))/np.pi

In [3]:
def findMax(q): #Finds the maximum height
	if q[3] < q[0] and q[3] < q[1] and q[3] < q[2] and q[3] < q[4] and q[3] < q[5] and q[3] < q[6]:
		return q[3]
	else:
		return 0

def findMin(q): #Finds the minimum height
	if q[3] > q[0] and q[3] > q[1] and q[3] > q[2] and q[3] > q[4] and q[3] > q[5] and q[3] > q[6]:
		return q[3]
	else:
		return 0


In [17]:
def compareTolerance(list_current, average_prev, tolerance):
    average_current = np.mean(np.asarray(list_current))
    difference = np.abs(average_current - average_prev)
    if difference < tolerance*average_current:
        return False
    else:
        return True
    

# Parameters

In [5]:
path_input = './exercises'
path_output = './output'
print_min = True
model_number = 101
scale_factor = 0.2
delay = 3
exercise = "pullup2"
video_format = ".avi"
font_face = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.5
font_color = (255,255,255)
body_points = {'rightShoulder':None,'rightWrist':None,'rightElbow':None,'leftShoulder':None,'leftWrist':None,'leftElbow':None}
body_points_score = {'rightShoulder':None,'rightWrist':None,'rightElbow':None,'leftShoulder':None,'leftWrist':None,'leftElbow':None}
body_angles = {'rightAngle':None,'leftAngle':None,'avgAngle':None}

# Image Analysis

In [ ]:
filenames = [f.path for f in os.scandir(path_input) if f.is_file() and f.path.endswith(('.png', '.jpg'))]

start = time.time()
with tf.Session() as sess:
    model_cfg, model_outputs = posenet.load_model(model_number, sess)
    output_stride = model_cfg['output_stride']
    for f in filenames:
        print(f)
        input_image, draw_image, output_scale = posenet.read_imgfile(
            f, scale_factor=scale_factor, output_stride=output_stride)
        heatmaps_result, offsets_result, displacement_fwd_result, displacement_bwd_result = sess.run(
            model_outputs,
            feed_dict={'image:0': input_image}
        )

        pose_scores, keypoint_scores, keypoint_coords = posenet.decode_multiple_poses(
            heatmaps_result.squeeze(axis=0),
            offsets_result.squeeze(axis=0),
            displacement_fwd_result.squeeze(axis=0),
            displacement_bwd_result.squeeze(axis=0),
            output_stride=output_stride,
            max_pose_detections=10,
            min_pose_score=0.25)

        keypoint_coords *= output_scale


        draw_image = posenet.draw_skel_and_kp(
            draw_image, pose_scores, keypoint_scores, keypoint_coords,
            min_pose_score=0.25, min_part_score=0.25)

        cv2.imwrite(os.path.join(path_output, os.path.relpath(f, path_input)), draw_image)

        
        print("Results for image: %s" % f)
        for pi in range(len(pose_scores)):
            if pose_scores[pi] == 0.:
                break
            print('Pose #%d, score = %f' % (pi, pose_scores[pi]))
            for ki, (s, c) in enumerate(zip(keypoint_scores[pi, :], keypoint_coords[pi, :, :])):
                #print('Keypoint %s, score = %f, coord = %s' % (posenet.PART_NAMES[ki], s, c))
                if posenet.PART_NAMES[ki] in body_points:
                    body_points[posenet.PART_NAMES[ki]] = c
                    body_points_score[posenet.PART_NAMES[ki]] = s
        #Calculate angle between shoulder and wrist using the elbow as the origin
        angle_right = angle((body_points['rightShoulder']-body_points['rightElbow']),(body_points['rightWrist']-body_points['rightElbow']))
        angle_left = angle((body_points['leftShoulder']-body_points['leftElbow']),(body_points['leftWrist']-body_points['leftElbow']))
        
        #Calculate the average confidence score of the shoulder, elbow, and wrist
        confidence_right = body_points_score['rightShoulder'] + body_points_score['rightElbow'] + body_points_score['rightWrist']
        confidence_left = body_points_score['leftShoulder'] + body_points_score['leftElbow'] + body_points_score['leftWrist']
        confidence_right /= 3
        confidence_left /= 3
        
        print('Angle Right: %f degrees with confidence %f' %(angle_right,confidence_right))
        print('Angle Left: %f degrees with confidence %f' %(angle_left,confidence_left))
        print()
    print('Average FPS:', len(filenames) / (time.time() - start))


# Video Analysis for Pushups and Pullups

In [21]:
with tf.Session() as sess:
        model_cfg, model_outputs = posenet.load_model(model_number, sess)
        output_stride = model_cfg['output_stride']
        cap = cv2.VideoCapture(exercise+video_format)
        fourcc = cv2.VideoWriter_fourcc('X','V','I','D')
        vid_out = cv2.VideoWriter('output '+exercise+video_format,fourcc, 30, (600,900), True)
        cap.set(3, 1080)
        cap.set(4, 1920)
        
        #create other queues to get more info
        moreInfoQ = deque()
        imageQ = deque()
        infoQ = deque()
        start = time.time()
        frame_count = 0
        picture_count = 1
        delay_count = 0
        avg_count = 10
        avg_counter = 0
        threshold = 0
        
        next_maxima = 'N/A'
        
        
        
        
        # Variables for tracking maxima
        m_var = {'Previous Average': 0, 'Initial Position': 0, 'Queue': deque(), 'First': True, 'Max Found': False,'Maxima Text':''}
        body = {'Right Eye':m_var, 'Nose':m_var}#'Left Eye':m_var,'Nose':m_var}
        body['Nose']['Previous Average'] = 0
        body['Nose']['Initial Position'] = 0
        body['Nose']['Queue'] = deque()
        body['Nose']['First'] = True
        tolerance = 0.2
        body['Nose']['Max Found'] = 0
        body['Nose']['Maxima Text'] = ''
        
        c_r = deque()
        c_l = deque()
        # maximum angles
        st = []
        # minimum angles
        sb = []
        while True:
            res, img = cap.read()
            if not res:
                break
            else:
                input_image, display_image, output_scale = posenet.process_input(img, scale_factor, output_stride)
            info_dict = {"Body Points": None, "Body Points Score": None, "Body Angles":None, "Scores Right": None, "Scores Left": None, "Image": None}
        
           
            #input_image, display_image, output_scale = posenet.read_cap(
              #  cap, scale_factor=0.2, output_stride=output_stride)
            
            heatmaps_result, offsets_result, displacement_fwd_result, displacement_bwd_result = sess.run(
                model_outputs,
                feed_dict={'image:0': input_image}
            )

            pose_scores, keypoint_scores, keypoint_coords = posenet.decode_multi.decode_multiple_poses(
                heatmaps_result.squeeze(axis=0),
                offsets_result.squeeze(axis=0),
                displacement_fwd_result.squeeze(axis=0),
                displacement_bwd_result.squeeze(axis=0),
                output_stride=output_stride,
                max_pose_detections=10,
                min_pose_score=0.15)

            keypoint_coords *= output_scale

            # TODO this isn't particularly fast, use GL for drawing and display someday...
            overlay_image = posenet.draw_skel_and_kp(
                display_image, pose_scores, keypoint_scores, keypoint_coords,
                min_pose_score=0, min_part_score=0)
            
            # CHANGE THE RESOLUTION OF THE OUTPUT VIDEO
            #overlay_image = cv2.resize(overlay_image, (1920, 1080))
            overlay_image = cv2.resize(overlay_image, (600, 900))
            
            # RESET VARIABLES FOR FOUND MAXIMA FOR EACH PART
            for body_part in body:
                body[body_part]['Max Found'] = False
            
            # APPEND THE IMAGE TO THE IMAGEQ
            imageQ.append(overlay_image)
            for pi in range(len(pose_scores)):
                if pose_scores[pi] == 0.:
                    break
                #logging.warning('Pose #%d, score = %f' % (pi, pose_scores[pi]))
                for ki, (s, c) in enumerate(zip(keypoint_scores[pi, :], keypoint_coords[pi, :, :])):
                    if posenet.PART_NAMES[ki] == "nose":
                        #print('Keypoint %s, score = %f, coord = %s' % (posenet.PART_NAMES[ki], s, c))
                        #adding coordinate to running queue
                        body['Nose']['Queue'].append(c[0])
                        #append more infomation to other queues here
                        
                        #if length equals 7, then will see if mid point is the min
                        if body['Nose']['Initial Position'] == 0:
                            body['Nose']['Initial Position'] = c[0]
                        if len(body['Nose']['Queue']) == 7:
                            if findMin(body['Nose']['Queue']) != 0 and compareTolerance(body['Nose']['Queue'],body['Nose']['Previous Average'], tolerance):
                                if body['Nose']['First'] == False or (body['Nose']['First'] == True and body['Nose']['Initial Position'] < c[0]):
                                    body['Nose']['Maxima Text'] = 'Minimum'
                                    body['Nose']['Previous Average'] = np.mean(np.asarray(body['Nose']['Queue']))
                                    body['Nose']['First'] = False
                                    body['Nose']['Max Found'] = True
                                    print('nose min')
                            if findMax(body['Nose']['Queue']) != 0 and compareTolerance(body['Nose']['Queue'],body['Nose']['Previous Average'], tolerance):
                                if body['Nose']['First'] == False or (body['Nose']['First'] == True and body['Nose']['Initial Position'] > c[0]):
                                    body['Nose']['Maxima Text'] = 'Maximum'
                                    body['Nose']['Previous Average'] = np.mean(np.asarray(body['Nose']['Queue']))
                                    body['Nose']['First'] = False
                                    body['Nose']['Max Found'] = True
                                    print('nose max')
                            body['Nose']['Queue'].popleft()
                    if posenet.PART_NAMES[ki] == "rightEye":
                        #print('Keypoint %s, score = %f, coord = %s' % (posenet.PART_NAMES[ki], s, c))
                        #adding coordinate to running queue
                        body['Right Eye']['Queue'].append(c[0])
                        #append more infomation to other queues here
                        
                        #if length equals 7, then will see if mid point is the min
                        if body['Right Eye']['Initial Position'] == 0:
                            body['Right Eye']['Initial Position'] = c[0]
                        if len(body['Right Eye']['Queue']) == 7:
                            if findMin(body['Right Eye']['Queue']) != 0 and compareTolerance(body['Right Eye']['Queue'],body['Right Eye']['Previous Average'], tolerance):
                                if body['Right Eye']['First'] == False or (body['Right Eye']['First'] == True and body['Right Eye']['Initial Position'] < c[0]):
                                    body['Right Eye']['Maxima Text'] = 'Minimum'
                                    body['Right Eye']['Previous Average'] = np.mean(np.asarray(body['Right Eye']['Queue']))
                                    body['Right Eye']['First'] = False
                                    body['Right Eye']['Max Found'] = True
                                    print('eye min')
                            if findMax(body['Right Eye']['Queue']) != 0 and compareTolerance(body['Right Eye']['Queue'],body['Right Eye']['Previous Average'], tolerance):
                                if body['Right Eye']['First'] == False or (body['Right Eye']['First'] == True and body['Right Eye']['Initial Position'] > c[0]):
                                    body['Right Eye']['Maxima Text'] = 'Maximum'
                                    body['Right Eye']['Previous Average'] = np.mean(np.asarray(body['Right Eye']['Queue']))
                                    body['Right Eye']['First'] = False
                                    body['Right Eye']['Max Found'] = True
                                    print('eye max')
                            body['Right Eye']['Queue'].popleft()
                    body_points[posenet.PART_NAMES[ki]] = c
                    body_points_score[posenet.PART_NAMES[ki]] = s
            
            # Check if there is a maxima in any of the points found
    
            
            #Calculate the average confidence score of the shoulder, elbow, and wrist
            confidence_right = body_points_score['rightShoulder'] + body_points_score['rightElbow'] + body_points_score['rightWrist']
            confidence_left = body_points_score['leftShoulder'] + body_points_score['leftElbow'] + body_points_score['leftWrist']
            confidence_right /= 3
            confidence_left /= 3
            angle_right = angle((body_points['rightShoulder']-body_points['rightElbow']),(body_points['rightWrist']-body_points['rightElbow']))
            angle_left = angle((body_points['leftShoulder']-body_points['leftElbow']),(body_points['leftWrist']-body_points['leftElbow']))
            weight_r = confidence_right / (confidence_right + confidence_left)
            weight_l = confidence_left / (confidence_right + confidence_left)
            weighted_average = (weight_r*angle_right + weight_l*angle_left)       
            body_angles['rightAngle'] = angle_right
            body_angles['leftAngle'] = angle_left
            body_angles['avgAngle'] = weighted_average
            
            if delay_count < delay:
                infoQ.append(info_dict)
                delay_count+=1
            else:
                oldQ = infoQ.popleft()
                infoQ.append(info_dict)
                
            if avg_counter < avg_count:    
                c_r.append(confidence_right)
                c_l.append(confidence_left)
                avg_counter += 1
            else:
                c_r.popleft()
                c_l.popleft()
                c_r.append(confidence_right)
                c_l.append(confidence_left)
            
            # MAKE IT SO WE CAN ONLY HAVE ALTERNATING MAXIMUM AND MINIMUM
            for body_part in body:
                if body[body_part]['Max Found'] == True:
                    if next_maxima == 'N/A' or body[body_part]['Maxima Text'] != next_maxima:
                        next_maxima = body[body_part]['Maxima Text']
                        maxima_found = True
                    else: 
                        maxima_found = False
            
            # Create video
            data_text_r = str('Angle Right: %f degrees with confidence %f' %(angle_right,confidence_right))
            data_text_l = str('Angle Left: %f degrees with confidence %f' %(angle_left,confidence_left))
            data_text_avg = str('Average Angle: %f' %(weighted_average))
            m_text = 'Number of maxima = %i ' %(picture_count-1)
            overlay_image = cv2.putText(overlay_image, data_text_r, (50,650), font_face, font_scale, font_color)
            overlay_image = cv2.putText(overlay_image, data_text_l, (50,700), font_face, font_scale, font_color)
            overlay_image = cv2.putText(overlay_image, data_text_avg, (50,750), font_face, font_scale, font_color)
            overlay_image = cv2.putText(overlay_image, str(maxima_found), (50,550), font_face, font_scale, font_color)
            vid_out.write(overlay_image)
            
            # STORE INFORMATION INTO DICTIONARY
            info_dict['Body Points'] = body_points
            info_dict['Body Points Score'] = body_points_score
            info_dict['Body Angles'] = body_angles
            info_dict['Scores Right'] = confidence_right
            info_dict['Scores Left'] = confidence_left
            info_dict['Image'] = overlay_image
            
            if print_min and maxima_found: 
                avg_r = 0
                avg_l = 0
                for i in np.arange(avg_count):
                    avg_r += c_r[i]
                    avg_l += c_l[i]
                avg_r /= avg_count
                avg_l /= avg_count
                if (avg_r > threshold and avg_l > threshold) or (oldQ['Scores Right'] > threshold and oldQ['Scores Left'] > threshold):
                    #Calculate angle between shoulder and wrist using the elbow as the origin
                    #angle_right = angle((oldQ['Body Points']['rightShoulder']-oldQ['Body Points']['rightElbow']),(oldQ['Body Points']['rightWrist']-oldQ['Body Points']['rightElbow']))
                    #angle_left = angle((oldQ['Body Points']['leftShoulder']-oldQ['Body Points']['leftElbow']),(oldQ['Body Points']['leftWrist']-oldQ['Body Points']['leftElbow']))
                    if next_maxima == 'Maximum':
                        st.append(angle_right)
                        st.append(angle_left)
                    else:
                        sb.append(angle_right)
                        sb.append(angle_left)
                    print('For image number: ', picture_count)
                    print(next_maxima, ' Height Found')
                    print('Angle Right: %f degrees with confidence %f and average confidence %f' %(oldQ['Body Angles']['rightAngle'],oldQ['Scores Right'],avg_r))
                    print('Angle Left: %f degrees with confidence %f and average confidence %f' %(oldQ['Body Angles']['leftAngle'],oldQ['Scores Left'],avg_l))
                    print()
                    oldQ['Image'] = cv2.putText(oldQ['Image'], next_maxima, (50,600), font_face, font_scale, font_color)
                    cv2.imwrite(r'C:\Users\micha\Desktop\ML\113dbspring19\posenet-python-master\posenet-python-master\output\ ' + body['Nose']['Maxima Text'] + ' ' + exercise + ' ' +  str(picture_count) + '.jpg',oldQ['Image'])
                    picture_count += 1
            
            cv2.imshow('posenet', overlay_image)
            frame_count += 1
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
                
        st = np.asarray(st)
        sb = np.asarray(sb)
        print('The maximum angle average is %f and the standard deviation is %f'% (np.mean(st),np.std(st)))
        print('The minimum angle average is %f and the standard deviation is %f'% (np.mean(sb),np.std(sb)))
        print(body['Nose']['Max Found'])
        vid_out.release()
        cap.release()
        cv2.destroyAllWindows();
        print('Average FPS: ', frame_count / (time.time() - start))

C:\Users\micha\Desktop\ML\113dbspring19\posenet-python-master\posenet-python-master\posenet\converter\config.py:9: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  cfg = yaml.load(cfg_f)


eye min
nose max
eye min
eye min
nose max
eye min
nose max
eye min
eye min
nose max
eye min
eye min
nose max
eye min
nose max
nose max
eye min
eye min
nose max
The maximum angle average is nan and the standard deviation is nan
The minimum angle average is nan and the standard deviation is nan
False
Average FPS:  0.010009387001711213


c:\users\micha\appdata\local\programs\python\python36\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
c:\users\micha\appdata\local\programs\python\python36\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
c:\users\micha\appdata\local\programs\python\python36\lib\site-packages\numpy\core\_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
c:\users\micha\appdata\local\programs\python\python36\lib\site-packages\numpy\core\_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
c:\users\micha\appdata\local\programs\python\python36\lib\site-packages\numpy\core\_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


# Video Analysis for Squat

In [ ]:
with tf.Session() as sess:
        model_cfg, model_outputs = posenet.load_model(model_number, sess)
        output_stride = model_cfg['output_stride']
        cap = cv2.VideoCapture(exercise)
        cap.set(3, 1080)
        cap.set(4, 1920)
        #coordinate queue
        runningQ = deque()
        #create other queues to get more info
        moreInfoQ = deque()
        imageQ = deque()
        infoQ = deque()
        start = time.time()
        frame_count = 0
        picture_count = 0
        delay_count = 0
        while True:
            res, img = cap.read()
            if not res:
                break
            else:
                input_image, display_image, output_scale = posenet.process_input(img, scale_factor, output_stride)
            info_dict = {"Body Points": None, "Body Points Score": None, "Scores Right": None, "Scores Left": None, "Image": None}
        
           
            #input_image, display_image, output_scale = posenet.read_cap(
              #  cap, scale_factor=0.2, output_stride=output_stride)
            
            heatmaps_result, offsets_result, displacement_fwd_result, displacement_bwd_result = sess.run(
                model_outputs,
                feed_dict={'image:0': input_image}
            )

            pose_scores, keypoint_scores, keypoint_coords = posenet.decode_multi.decode_multiple_poses(
                heatmaps_result.squeeze(axis=0),
                offsets_result.squeeze(axis=0),
                displacement_fwd_result.squeeze(axis=0),
                displacement_bwd_result.squeeze(axis=0),
                output_stride=output_stride,
                max_pose_detections=10,
                min_pose_score=0.15)

            keypoint_coords *= output_scale

            # TODO this isn't particularly fast, use GL for drawing and display someday...
            overlay_image = posenet.draw_skel_and_kp(
                display_image, pose_scores, keypoint_scores, keypoint_coords,
                min_pose_score=0.15, min_part_score=0.1)
            
            #overlay_image = cv2.resize(overlay_image, (1024, 960))
            overlay_image = cv2.resize(overlay_image, (600, 900))
            minimum_found = False

            imageQ.append(overlay_image)
            for pi in range(len(pose_scores)):
                if pose_scores[pi] == 0.:
                    break
                #logging.warning('Pose #%d, score = %f' % (pi, pose_scores[pi]))
                for ki, (s, c) in enumerate(zip(keypoint_scores[pi, :], keypoint_coords[pi, :, :])):
                    if posenet.PART_NAMES[ki] == "leftEye":
                        #print('Keypoint %s, score = %f, coord = %s' % (posenet.PART_NAMES[ki], s, c))
                        #adding coordinate to running queue
                        runningQ.append(c[0])
                        #append more infomation to other queues here
                        
                        #if length equals 7, then will see if mid point is the min
                        if len(runningQ) == 7:
                            if findMin(runningQ) != 0:
                                #print('FOUND MIN')
                                minimum_found = True
                                #print(findMin(runningQ))
                            runningQ.popleft()
                            #popleft to all other queues you create for more info here
                    if posenet.PART_NAMES[ki] in body_points:
                        body_points[posenet.PART_NAMES[ki]] = c
                        body_points_score[posenet.PART_NAMES[ki]] = s
                        
                            
                        
            info_dict['Body Points'] = body_points
            info_dict['Body Points Score'] = body_points_score
            
            #Calculate the average confidence score of the shoulder, elbow, and wrist
            confidence_right = body_points_score['rightShoulder'] + body_points_score['rightElbow'] + body_points_score['rightWrist']
            confidence_left = body_points_score['leftShoulder'] + body_points_score['leftElbow'] + body_points_score['leftWrist']
            confidence_right /= 3
            confidence_left /= 3
            info_dict['Scores Right'] = confidence_right
            info_dict['Scores Left'] = confidence_left
            info_dict['Image'] = overlay_image
            
            if delay_count < delay:
                infoQ.append(info_dict)
                delay_count+=1
            else:
                oldQ = infoQ.popleft()
                infoQ.append(info_dict)
                
            if print_min and minimum_found:
                for i in np.arange(3):
                    avg_r += infoQ[i]['']
                #Calculate angle between shoulder and wrist using the elbow as the origin
                angle_right = angle((oldQ['Body Points']['rightShoulder']-oldQ['Body Points']['rightElbow']),(oldQ['Body Points']['rightWrist']-oldQ['Body Points']['rightElbow']))
                angle_left = angle((oldQ['Body Points']['leftShoulder']-oldQ['Body Points']['leftElbow']),(oldQ['Body Points']['leftWrist']-oldQ['Body Points']['leftElbow']))
                print('Maximum Height Found')
                print('Angle Right: %f degrees with confidence %f' %(angle_right,confidence_right))
                print('Angle Left: %f degrees with confidence %f' %(angle_left,confidence_left))
                print()
                cv2.imwrite(r'C:\Users\micha\Desktop\ML\113dbspring19\posenet-python-master\posenet-python-master\output\maximum_squat' + exercise + str(picture_count) + '.jpg',oldQ['Image'])

                picture_count += 1
            cv2.imshow('posenet', overlay_image)
            frame_count += 1
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows();
        print('Average FPS: ', frame_count / (time.time() - start))

In [ ]:
avg_r = 0
for i in np.arange(4):
    print(infoQ[i]['Scores Right'])
    avg_r += infoQ[i]['Scores Right']

In [ ]:
if posenet.PART_NAMES[ki] == "rightEye":
                        #print('Keypoint %s, score = %f, coord = %s' % (posenet.PART_NAMES[ki], s, c))
                        #adding coordinate to running queue
                        body['Right Eye']['Queue'].append(c[0])
                        #append more infomation to other queues here
                        
                        #if length equals 7, then will see if mid point is the min
                        if body['Right Eye']['Initial Position'] == 0:
                            body['Right Eye']['Initial Position'] = c[0]
                        if len(body['Right Eye']['Queue']) == 7:
                            if findMin(body['Right Eye']['Queue']) != 0 and compareTolerance(body['Right Eye']['Queue'],body['Right Eye']['Previous Average'], tolerance):
                                if body['Right Eye']['First'] == False or (body['Right Eye']['First'] == True and body['Right Eye']['Initial Position'] < c[0]):
                                    maxima_found = True
                                    body['Right Eye']['Maxima Text'] = 'Minimum'
                                    body['Right Eye']['Previous Average'] = np.mean(np.asarray(body['Right Eye']['Queue']))
                                    body['Right Eye']['First'] = False
                                    
                                    body['Right Eye']['Max Count'] += 1
                            if findMax(body['Right Eye']['Queue']) != 0 and compareTolerance(body['Right Eye']['Queue'],body['Right Eye']['Previous Average'], tolerance):
                                if body['Right Eye']['First'] == False or (body['Right Eye']['First'] == True and body['Right Eye']['Initial Position'] > c[0]):
                                    maxima_found = True
                                    body['Right Eye']['Maxima Text'] = 'Maximum'
                                    body['Right Eye']['Previous Average'] = np.mean(np.asarray(body['Right Eye']['Queue']))
                                    body['Right Eye']['First'] = False
                                    body['Right Eye']['Max Count'] += 1
                            body['Right Eye']['Queue'].popleft()